In [3]:
# Importing the libraries
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPooling2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
# ResNet50 model is trained on 224x224 images of ImageNet dataset
from tensorflow.keras.applications import ResNet50


In [ ]:
# Implementing Convolution Block
# Relu activation is used after every convolution layer
# Batch Normalization is used after every convolution layer as it improves the training speed and reduces the chances of getting stuck in local minima

def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding='same')(inputs)
    # Batch Normalization is used after every convolution layer as it improves the training speed and reduces the chances of getting stuck in local minima
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # Changing inputs to x so that we can add the inputs to the output of the convolution block
    x = Conv2D(num_filters, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    return x